In [34]:
# Importamos las librerías necesarias

# Tratamiento de datos
import pandas as pd
import numpy as np

# Librería para convertir palabras a numeros
from word2number import w2n

# Configuración para poder visualizar todas las columnas y filas de los DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Fase 2: Transformación de los datos.

In [35]:
# Leo archivo CSV
hr_raw = pd.read_csv("HR RAW DATA.csv", index_col=0)

# Lo guardo en un DataFrame
df_hr_raw = pd.DataFrame(hr_raw)

# Hago una copia del DataFrame que iré limpiando
df_limpia = df_hr_raw.copy()

## A. Limpieza, normalización y transformación de datos

Este paso incluye la limpieza de datos, la normalización y la conversión de tipos de datos. Tras revisar el csv con los datos en bruto hemos podido observar qué acciones son necesarias llevar a cabo:

- ✔️Estandarizar strings
    - `BusinessTravel` (object) tiene guiones medios y bajos, estandarizar a guion bajo
    - `MaritalStatus` (object) revisar valores con faltas ortograficas
- ✔️Revisar que los que tienen que ser numericos no son string u otro tipo
    - `Age` (object) cambiar object a valores numéricos (por ej: thirty -> 30)
    - `DailyRate` (object) quitar simbolo dolar y pasar a valor numerico. Tener en cuenta que hay strings que ponen `'nan'`
    - `HourlyRate` (object) revisar que hacer con los string `'Not available'`
    - `MonthlyIncome`  (object) revisar que hacer con los string 'nan', ver si poner FLOAT o INT
    - `PerformanceRating`  (object) revisar que hacer con los string 'nan'
    - `TOTALWORKINGYEARS` (object) revisar que hacer con los string 'nan'
    - `WORKLIFEBALANCE` (object) revisar que hacer con los string 'nan'
    - `YearsInCurrentRole` (object) revisar que hacer con los string 'nan'
- ✔️Estandarizar valores numericos
    - `DistanceFromHome`(int64) tiene valores negativos, quitar
- ✔️Estandarizar booleanos o valores binarios
    - `Gender` (int64) cambiar 0 y 1 a algo mas legible
    - `RemoteWork` (object) estandarizar valores
- Revisar que hacer con los string 'nan', campos vacíos o nulos (Fase 2B)
    - Columnas anteriores
    - `employeenumber` (object) revisar que hacer con los string 'nan'

### 1. Primer filtro de eliminación de columnas

Elimino las columnas que no aportan información relevante o correcta antes de la primera limpieza, ya que la labor de limpieza puede ser bastante tediosa para ciertas columnas que no son necesarias. Las columnas a revisar antes de eliminar son:

- `Department` pocos datos, revisar si eliminar (MANTENER DE MOMENTO)
- `employeecount` todos con 1
- `Over18` solo hay Y y vacíos, revisar si es necesaria (ya hay una columna con la edad `Age`)
- `StandardHours` son 80 o nan, revisar si es necesaria
- `Salary` tiene un único valor para todos los registros, 1000000000$
- `RoleDepartament` eliminar, información redundantes (ya está el Role y el Department en otras columnas)
- `NUMBERCHILDREN` todas vacías, eliminar
- `YearsInCurrentRole` alto porcentaje de nulos
- `SameAsMonthlyIncome` igual que `MonthlyIncome`

Iremos una por una analizando los datos que hay en cada columna para confirmar si hay que eliminarlas o no.

- `employeecount`: todos los employees son únicos, no se necesita un conteo. La columna no aporta ninguna informacion relevante por lo que se podría eliminar.

In [36]:
print(f"Los valores únicos de la columna 'employeecount' son: {df_limpia['employeecount'].unique()}")
print(f"Y el porcentaje de nulos es: {df_limpia['employeecount'].isnull().sum() / df_limpia.shape[0]}")

Los valores únicos de la columna 'employeecount' son: [1]
Y el porcentaje de nulos es: 0.0


- `Over18`: La edad mínima en la columna `Age` es 18 y no tiene nulos, por lo que todos los employees son mayores de edad. La columna `Over18` debería de tener el valor `Y` (yes) para todas las filas, pero no aporta ninguna información relevante, por lo que también se podría eliminar.

In [37]:
print(f"Los valores únicos de la columna 'Over18' son: {df_limpia['Over18'].unique()}")
print(f"Y el porcentaje de nulos de 'Over18' es: {round(df_limpia['Over18'].isnull().sum() / df_limpia.shape[0],2)}")
print(f"La edad mínima en la columna 'Age' es: {df_limpia['Age'].min()}")
print(f"Y el porcentaje de nulos de 'Age' es: {round(df_limpia['Age'].isnull().sum() / df_limpia.shape[0],2)}")

Los valores únicos de la columna 'Over18' son: ['Y' nan]
Y el porcentaje de nulos de 'Over18' es: 0.56
La edad mínima en la columna 'Age' es: 18
Y el porcentaje de nulos de 'Age' es: 0.0


- `StandardHours`: El porcentaje de nulos en esta columna es muy alta y el único valor que tiene diferente al nulo es de 80. Esta columna no aporta información suficiente para poder imputarla, la vamos a eliminar.

In [38]:
print(f"Los valores únicos de la columna 'StandardHours' son: {df_limpia['StandardHours'].unique()}")
print(f"Y el porcentaje de nulos de 'StandardHours' es: {round(df_limpia['StandardHours'].isnull().sum() / df_limpia.shape[0],2)}")

Los valores únicos de la columna 'StandardHours' son: [nan '80,0']
Y el porcentaje de nulos de 'StandardHours' es: 0.74


- `Salary`: Tiene un unico valor, 1000000000$. Es un valor demasiado elevado y es el mismo para todos los employees (no hay nulos), por lo que deducimos que son valores erróneos. Eliminaríamos esta columna también por incongruencia.

In [39]:
print(f"Los valores únicos de la columna 'Salary' son: {df_limpia['Salary'].unique()}")
print(f"Y el porcentaje de nulos de 'Salary' es: {round(df_limpia['Salary'].isnull().sum() / df_limpia.shape[0],2)}")

Los valores únicos de la columna 'Salary' son: ['1000000000$']
Y el porcentaje de nulos de 'Salary' es: 0.0


- `RoleDepartament`: Esta columna repite la misma información que `Department` y `JobRole` y las combina (nos dan la misma información expresada de forma diferente), pero tiene un alto porcentaje de nulos, el mismo que `Department`. La información que aporta es redundante e incluso insuficiente por la cantidad de nulos, así que también la eliminaríamos.

In [40]:
print(f"Y el porcentaje de nulos de 'RoleDepartament' es: {round(df_limpia['RoleDepartament'].isnull().sum() / df_limpia.shape[0],2)}")
print(f"Es el mismo porcentaje nulos que 'Department' es: {round(df_limpia['Department'].isnull().sum() / df_limpia.shape[0],2)}\n")
print(f"Un ejemplo de datos repetidos sería la fila 1428\n{df_limpia.loc[1428,['JobRole','Department','RoleDepartament']]}\n")
print(f"Un ejemplo de datos nulos sería la fila 0\n{df_limpia.loc[0,['JobRole','Department','RoleDepartament']]}")

Y el porcentaje de nulos de 'RoleDepartament' es: 0.81
Es el mismo porcentaje nulos que 'Department' es: 0.81

Un ejemplo de datos repetidos sería la fila 1428
JobRole                       SalES eXecUTiVE 
Department                              Sales 
RoleDepartament     SalES eXecUTiVE  -  Sales 
Name: 1428, dtype: object

Un ejemplo de datos nulos sería la fila 0
JobRole             resEArch DIREcToR 
Department                         NaN
RoleDepartament                    NaN
Name: 0, dtype: object


- `NUMBERCHILDREN`: Todos los valores de esta columna son nulos, no aporta ninguna información por lo que la eliminaríamos.

In [41]:
print(f"El porcentaje de nulos de 'NUMBERCHILDREN' es: {round(df_limpia['NUMBERCHILDREN'].isnull().sum() / df_limpia.shape[0],2)}")

El porcentaje de nulos de 'NUMBERCHILDREN' es: 1.0


- `YearsInCurrentRole` alto porcentaje de nulos, no aporta información suficiente para imputarla, la eliminaríamos.

In [42]:
print(f"El porcentaje de nulos de 'YearsInCurrentRole' es: {round(df_limpia['YearsInCurrentRole'].isnull().sum() / df_limpia.shape[0],2)}")

El porcentaje de nulos de 'YearsInCurrentRole' es: 0.98


- `SameAsMonthlyIncome`: Todos los valores de esta columna son iguales a los de `MonthlyIncome`, la información es redundante y se podría eliminar.

In [43]:
# Comparo las columnas 'SameAsMonthlyIncome' y 'MonthlyIncome' para comprobar si son iguales
son_iguales = df_limpia['SameAsMonthlyIncome'].equals(df_limpia['MonthlyIncome'])
print(f"Las columnas 'SameAsMonthlyIncome' y 'MonthlyIncome' son iguales: {son_iguales}")

Las columnas 'SameAsMonthlyIncome' y 'MonthlyIncome' son iguales: True


En este primer filtro elimino las columnas anteriormente analizadas, ya que he comprobado que son redundantes o no aportan información suficiente.

In [44]:
columnas_a_eliminar = ['employeecount','Over18','StandardHours','Salary','RoleDepartament','NUMBERCHILDREN','YearsInCurrentRole','SameAsMonthlyIncome']

# Elimino las columnas de df_limpia, ya que es la copia del DataFrame que estoy limpiando
df_limpia.drop(columnas_a_eliminar, axis = 1, inplace = True)
df_limpia.head(3)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,DateBirth,RemoteWork
0,51,No,NaN,"684,0$",NaN,6,3,NaN,"162,0",1,0,51,3,5,resEArch DIREcToR,3,NaN,"19537,0",6462,7,No,13,"3,0",3,0,NaN,5,"3,0",20,15,15,1972,Yes
1,52,No,NaN,"699,0$",NaN,1,4,Life Sciences,"259,0",3,0,65,2,5,ManAGeR,3,NaN,"19999,0",5678,0,NaN,14,"3,0",1,1,"34,0",5,"3,0",33,11,9,1971,1
2,42,No,travel_rarely,"532,0$",Research & Development,4,2,Technical Degree,"319,0",3,0,58,3,5,ManaGER,4,Married,"19232,0",4933,1,No,11,"3,0",4,0,"22,0",3,NaN,22,11,15,1981,1


### 2. Normalizamos el nombre de las columnas

In [45]:
nombres_snake_case = {
    'employeenumber': 'EmployeeNumber',
    'NUMCOMPANIESWORKED': 'NumCompaniesWorked',
    'TOTALWORKINGYEARS': 'TotalWorkingYears',
    'WORKLIFEBALANCE': 'WorkLifeBalance',
    'YEARSWITHCURRMANAGER': 'YearsWithCurrManager',
}

# Renombro las columnas para que todas estén en SnakeCase y sobreescribo el DataFrame usando el método rename
df_limpia.rename(columns=nombres_snake_case, inplace=True)
df_limpia.sample(3)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,DateBirth,RemoteWork
1588,28,Yes,NaN,"654,0$",Research & Development,1,2,NaN,NaN,1,1,67,1,1,reSEArCh sCieNtiST,2,NaN,"2216,0",3872,7,Yes,13,"3,0",4,0,"10,0",4,"3,0",7,3,7,1995,Yes
821,59,No,non-travel,"1420,0$",NaN,2,4,Human Resources,NaN,3,1,32,2,5,MAnAGER,4,NaN,"18844,0",21922,9,No,21,"4,0",4,1,NaN,3,"3,0",3,2,2,1964,0
681,34,No,NaN,"878,0$",NaN,10,4,NaN,"277,0",4,0,43,3,1,reseARCh SciEnTiST,3,Divorced,"3815,0",5972,1,Yes,17,"3,0",4,1,NaN,4,"4,0",5,2,0,1989,Yes


### 3. Normalizamos los valores de las columnas

#### 3.1 Estandarización valores numéricos

##### 3.1.1 Columnas con valores categóricos que deberían de ser numéricas



- `Age`                          object -> num
- `DailyRate`                    object -> num
- `HourlyRate`                   object -> num
- `MonthlyIncome`                object-> num
- `PerformanceRating`            object -> num
- `TOTALWORKINGYEARS`            object -> num
- `WORKLIFEBALANCE`              object -> num
- `YearsInCurrentRole`           object -> num

- `Age`: Esta columna debería de ser numérica pero contiene algunos valores que son strings (numeros escritos). Creo una función para convertir las palabras a números, para ello me he instalado la librería `from word2number import w2n`

In [46]:
print(f"Los valores únicos de la columna 'Age' son: {df_limpia['Age'].unique()}\n")
print(f"Y los valores nulos que hay en la columna son: {df_limpia['Age'].isnull().sum()}")

Los valores únicos de la columna 'Age' son: ['51' '52' '42' '47' '46' '48' '59' '41' '56' '38' '55' '40' '58' '35'
 '45' '33' '36' '34' 'forty-seven' '53' '43' '60' '32' '37' '49' '39' '50'
 '44' '30' 'fifty-eight' '29' '31' '54' '57' '27' 'thirty-six' '28' '26'
 'fifty-five' '25' 'fifty-two' 'thirty-one' '24' 'thirty' '23' '22' '21'
 '20' 'twenty-six' '19' 'thirty-seven' '18' 'thirty-two' 'twenty-four']

Y los valores nulos que hay en la columna son: 0


In [47]:
# Función para convertir palabras a números
def palabra_a_numero(age):
    
    # Intento convertir palabra a numero
    try:
        return w2n.word_to_num(age)
    # Si me da error, es que ya es un numero que se puede convertir de str a int
    except ValueError:
        return int(age)
        
# Aplicamos la función a la columna 'Age'
df_limpia['Age'] = df_limpia['Age'].apply(palabra_a_numero)

# Verificamos que ya no hay palabras
print(df_limpia['Age'].unique())

[51 52 42 47 46 48 59 41 56 38 55 40 58 35 45 33 36 34 53 43 60 32 37 49
 39 50 44 30 29 31 54 57 27 28 26 25 24 23 22 21 20 19 18]


- `DailyRate`, `HourlyRate`, `MonthlyIncome`, `TotalWorkingYears`, `WorkLifeBalance`, `PerformanceRating` son columnas que incluyen valores numericos decimales pero que aparecen como tipo string en el DataFrame. Estos valores se pueden convertir en float. Habría que sustituir la coma por un punto y gestionar los 'nan' que son string para convertirlos en NaN.

    Hay que tener en cuenta ciertas particularidades, ya que `DailyRate` tiene el símbolo dólar y los valores nulos de `HourlyRate` aparecen como 'Not available', 

In [48]:
print(f"Los valores únicos de la columna 'DailyRate' son strings: {df_limpia['DailyRate'].unique()}")
print(f"Los valores únicos de la columna 'HourlyRate' son strings: {df_limpia['HourlyRate'].unique()}")
print(f"Los valores únicos de la columna 'MonthlyIncome' son strings: {df_limpia['MonthlyIncome'].unique()}")
print(f"Los valores únicos de la columna 'PerformanceRating' son strings: {df_limpia['PerformanceRating'].unique()}\n")
print(f"Los valores únicos de la columna 'TotalWorkingYears' son strings: {df_limpia['TotalWorkingYears'].unique()}\n")
print(f"Los valores únicos de la columna 'WorkLifeBalance' son strings: {df_limpia['WorkLifeBalance'].unique()}")

Los valores únicos de la columna 'DailyRate' son strings: ['684,0$' '699,0$' '532,0$' '359,0$' '1319,0$' '117,0$' '1435,0$' '635,0$'
 '1276,0$' '840,0$' '247,0$' '1369,0$' '201,0$' '1360,0$' '692,0$'
 '1398,0$' '286,0$' '1402,0$' '819,0$' '884,0$' '1238,0$' '515,0$'
 '1223,0$' '202,0$' '928,0$' '607,0$' '266,0$' '429,0$' '589,0$' 'nan$'
 '1180,0$' '1282,0$' '776,0$' '665,0$' '526,0$' '1034,0$' '1403,0$'
 '1499,0$' '580,0$' '859,0$' '263,0$' '1376,0$' '885,0$' '1003,0$'
 '1321,0$' '394,0$' '1372,0$' '1333,0$' '228,0$' '737,0$' '823,0$'
 '667,0$' '301,0$' '573,0$' '1329,0$' '630,0$' '1063,0$' '1017,0$'
 '1296,0$' '939,0$' '1355,0$' '1448,0$' '200,0$' '1202,0$' '404,0$'
 '208,0$' '813,0$' '465,0$' '1189,0$' '1001,0$' '1394,0$' '161,0$'
 '288,0$' '682,0$' '1354,0$' '147,0$' '119,0$' '1413,0$' '452,0$' '334,0$'
 '1132,0$' '982,0$' '480,0$' '1099,0$' '672,0$' '1379,0$' '583,0$'
 '1492,0$' '1050,0$' '469,0$' '237,0$' '1440,0$' '1291,0$' '1157,0$'
 '1336,0$' '1224,0$' '735,0$' '1389,0$' '638,0

In [49]:
def objeto_a_numero(cadena):
    if isinstance(cadena, str):
        cadena = cadena.replace('$', '').replace(',', '.')  # Reemplazar comas por puntos
        if cadena == 'nan' or cadena == 'Not Available':  # Manejar el caso de 'nan'
            return np.nan
        else:
            return float(cadena)  # Convertir a float y luego a int si no es 'nan'
    else:
        return cadena  # Devolver el valor original si no es una cadena
    
# Guardo en una lista las columnas que quiero convertir a enteros
col_a_enteros = ['DailyRate','HourlyRate','MonthlyIncome','TotalWorkingYears','WorkLifeBalance','PerformanceRating']

# Recorro la lista para aplicar a cada columna la funcion
for col in col_a_enteros:
    df_limpia[col] = df_limpia[col].apply(objeto_a_numero)
    
    # Si quisiera convertirlos a int podría hacer:
    # df_limpia[col] = df_limpia[col].apply(objeto_a_numero).astype('Int64')
    # En pandas, el tipo Int64 es un tipo de entero que soporta valores nulos (NaN), y cuando se usa este tipo,
    # los valores nulos se representan como <NA> en lugar de NaN. 

In [50]:
df_limpia[col_a_enteros].dtypes


DailyRate            float64
HourlyRate           float64
MonthlyIncome        float64
TotalWorkingYears    float64
WorkLifeBalance      float64
PerformanceRating    float64
dtype: object

In [51]:
df_limpia[col_a_enteros].sample(5)

,DailyRate,HourlyRate,MonthlyIncome,TotalWorkingYears,WorkLifeBalance,PerformanceRating
1227,1324.0,81.0,NaN,12.0,2.0,NaN
150,282.0,58.0,19187.0,23.0,3.0,NaN
351,657.0,66.0,4335.0,11.0,2.0,3.0
1486,430.0,40.0,NaN,23.0,3.0,3.0
1117,1054.0,67.0,3445.0,6.0,2.0,3.0


##### 3.1.2 Valores numéricos inconsistentes


- La columna `DistanceFromHome` tiene valores negativos.

In [52]:
print(f"Los valores únicos de la columna 'DistanceFromHome' incluyen algunos negativos: {df_limpia['DistanceFromHome'].unique()}\n")
print(f"Los valores nulos de la columna 'DistanceFromHome' son: {df_limpia['DistanceFromHome'].isnull().sum()}")

Los valores únicos de la columna 'DistanceFromHome' incluyen algunos negativos: [  6   1   4   2   3  22  25   9   7  23  10  12  14 -13  15   8 -42  28
 -37   5  16 -35  26 -26  24  29 -25  17  21 -18 -10 -30 -27  20 -31 -29
 -39  18 -21 -15  11  13 -14  19 -33 -34 -46 -36 -19  27 -12 -23 -45 -28
 -47 -32 -24 -16 -22 -41 -49 -11 -48 -38 -20 -17 -43 -40 -44]

Los valores nulos de la columna 'DistanceFromHome' son: 0


In [53]:
# Si la columna contiene valores numéricos, conviértelos a positivos si son negativos
df_limpia['DistanceFromHome'] = df_limpia['DistanceFromHome'].abs()

# Verifico si hay valores negativos en la columna 'DistanceFromHome'
hay_negativos = (df_limpia['DistanceFromHome'] < 0).any()

if hay_negativos:
    print("La columna 'DistanceFromHome' contiene valores negativos.")
else:
    print("La columna 'DistanceFromHome' no contiene valores negativos.")

La columna 'DistanceFromHome' no contiene valores negativos.


In [54]:
print(f"Después de limpiarla, los valores únicos de la columna 'DistanceFromHome' son: {df_limpia['DistanceFromHome'].unique()}\n")

Después de limpiarla, los valores únicos de la columna 'DistanceFromHome' son: [ 6  1  4  2  3 22 25  9  7 23 10 12 14 13 15  8 42 28 37  5 16 35 26 24
 29 17 21 18 30 27 20 31 39 11 19 33 34 46 36 45 47 32 41 49 48 38 43 40
 44]



In [55]:
# Eliminar los valores negativos de la columna DistanceFromHome
#df_limpia['DistanceFromHome'] = df_limpia['DistanceFromHome'].apply(lambda x: x if x >= 0 else np.nan)

# Convertir DistanceFromHome a entero
#df_limpia['DistanceFromHome'] = df_limpia['DistanceFromHome'].astype('Int64')

#### 3.2 Estandarizar booleanos o valores binarios


- `Gender` (int64) cambiar 0 y 1 a algo mas legible
- `RemoteWork` (object) estandarizar valores

- `Gender`: La columna `Gender` tiene valores de 0 y 1, los cuales son pocos intutitivos. Los reemplazo por "Male" y "Female", o "M" y "F" por ejemplo.

In [56]:
print(f"Los valores únicos de la columna 'Gender' son: {df_limpia['Gender'].unique()}")

Los valores únicos de la columna 'Gender' son: [0 1]


In [57]:
# Reemplazo los valores de la columna Gender por female y male
df_limpia['Gender'] = df_limpia['Gender'].replace({0: 'female', 1: 'male'})
print(f"Después de la limpieza, los valores únicos de la columna 'Gender' son: {df_limpia['Gender'].unique()}")

Después de la limpieza, los valores únicos de la columna 'Gender' son: ['female' 'male']


- `RemoteWork`

In [58]:
# Habría que estandarizar los datos 1/0, 'yes'/'no', 'false'/'true' del atributo RemoteWork
print(f"Los valores únicos de la columna 'RemoteWork' son: {df_limpia['RemoteWork'].unique()}")

Los valores únicos de la columna 'RemoteWork' son: ['Yes' '1' 'False' '0' 'True']


In [59]:
# Habría que estandarizar los datos 1/0, 'yes'/'no', 'false'/'true' del atributo RemoteWork
# Para una mayor legibilidad usamos los valores True/False en Remote Work
rw_datos_limpios = {
    'Yes': 'yes',
    '1': 'yes',
    '0': 'no',
    'True': 'yes',
    'False': 'no'
}

df_limpia['RemoteWork'] = df_limpia['RemoteWork'].replace(rw_datos_limpios)
print(f"Después de la limpieza, los valores únicos de la columna 'RemoteWork' son: {df_limpia['RemoteWork'].unique()}")

Después de la limpieza, los valores únicos de la columna 'RemoteWork' son: ['yes' 'no']


#### 3.3 Estandarización strings


- `BusinessTravel` (object) tiene guiones medios y bajos, estandarizar a guion bajo
- `Department`(object) tiene espacios al principio y al final, así como símbolo como '&'
- `JobRole`(object) iene espacios al principio y al final, y también letras en minusculas y en mayusculas
- `MaritalStatus` (object) revisar valores con faltas ortograficas

In [60]:
print(f"Los valore únicos de la columna 'Attrition' son: {df_limpia['Attrition'].unique()}\n")
print(f"Los valore únicos de la columna 'BusinessTravel' son: {df_limpia['BusinessTravel'].unique()}\n")
print(f"Los valore únicos de la columna 'Department' son: {df_limpia['Department'].unique()}\n")
print(f"Los valore únicos de la columna 'EducationField' son: {df_limpia['EducationField'].unique()}\n")
print(f"Los valore únicos de la columna 'Gender' son: {df_limpia['Gender'].unique()}\n")
print(f"Los valore únicos de la columna 'JobRole' son: {df_limpia['JobRole'].unique()}\n")
print(f"Los valore únicos de la columna 'MaritalStatus' son: {df_limpia['MaritalStatus'].unique()}\n")

Los valore únicos de la columna 'Attrition' son: ['No' 'Yes']

Los valore únicos de la columna 'BusinessTravel' son: [nan 'travel_rarely' 'travel_frequently' 'non-travel']

Los valore únicos de la columna 'Department' son: [nan ' Research & Development ' ' Sales ' ' Human Resources ']

Los valore únicos de la columna 'EducationField' son: [nan 'Life Sciences' 'Technical Degree' 'Medical' 'Other' 'Marketing'
 'Human Resources']

Los valore únicos de la columna 'Gender' son: ['female' 'male']

Los valore únicos de la columna 'JobRole' son: [' resEArch DIREcToR ' ' ManAGeR ' ' ManaGER ' ... ' sAlES ExECUTivE '
 ' SaLes ExecUtIVe ' ' mAnUfactURInG DiRECTOr ']

Los valore únicos de la columna 'MaritalStatus' son: [nan 'Married' 'Divorced' 'Single' 'divorced' 'Marreid']



In [61]:
# Función para reemplazar guiones medios y los espacios por guiones bajos, poner todo en minuscula y sustituir el simbolo '&' por 'and'
def estandarizar_texto(value):
    if isinstance(value, str):
        return value.lower().strip().replace('-', '_').replace(' ','_').replace('&','and')
    return value

# Aplicar la función a todas las columnas
df_limpia= df_limpia.applymap(estandarizar_texto)

# Verificar el resultado
df_limpia.tail()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,DateBirth,RemoteWork
1609,36,yes,travel_rarely,530.0,NaN,3,1,life_sciences,"967,0",3,female,51.0,2,3,sales_executive,4,married,10325.0,5518,1,NaN,11,NaN,1,1,NaN,6,3.0,16,3,7,1987,no
1610,45,no,non_travel,805.0,NaN,4,2,NaN,"972,0",3,female,57.0,3,2,laboratory_technician,2,NaN,4447.0,23163,1,NaN,12,3.0,2,0,NaN,5,2.0,9,0,8,1978,yes
1611,39,no,travel_rarely,903.0,NaN,13,5,NaN,NaN,13,female,41.0,4,3,sales_executive,3,single,NaN,2560,0,no,18,3.0,4,0,9.0,3,3.0,8,0,7,1984,yes
1612,36,no,non_travel,1229.0,NaN,8,4,technical_degree,"990,0",1,female,84.0,3,2,sales_executive,4,divorced,NaN,25952,4,no,13,NaN,4,2,12.0,3,3.0,7,0,7,1987,yes
1613,46,no,NaN,566.0,NaN,7,2,medical,NaN,4,female,75.0,3,3,manufacturing_director,3,NaN,10845.0,24208,6,NaN,13,3.0,2,1,NaN,3,3.0,8,0,7,1977,no


In [62]:
print(f"Después de la limpieza, los valores únicos de la columna 'Attrition' son: {df_limpia['Attrition'].unique()}\n")
print(f"Los valore únicos de la columna 'BusinessTravel' son: {df_limpia['BusinessTravel'].unique()}\n")
print(f"Los valore únicos de la columna 'Department' son: {df_limpia['Department'].unique()}\n")
print(f"Los valore únicos de la columna 'EducationField' son: {df_limpia['EducationField'].unique()}\n")
print(f"Los valore únicos de la columna 'Gender' son: {df_limpia['Gender'].unique()}\n")
print(f"Los valore únicos de la columna 'JobRole' son: {df_limpia['JobRole'].unique()}\n")
print(f"Los valore únicos de la columna 'MaritalStatus' son: {df_limpia['MaritalStatus'].unique()}\n")

Después de la limpieza, los valores únicos de la columna 'Attrition' son: ['no' 'yes']

Los valore únicos de la columna 'BusinessTravel' son: [nan 'travel_rarely' 'travel_frequently' 'non_travel']

Los valore únicos de la columna 'Department' son: [nan 'research_and_development' 'sales' 'human_resources']

Los valore únicos de la columna 'EducationField' son: [nan 'life_sciences' 'technical_degree' 'medical' 'other' 'marketing'
 'human_resources']

Los valore únicos de la columna 'Gender' son: ['female' 'male']

Los valore únicos de la columna 'JobRole' son: ['research_director' 'manager' 'sales_executive' 'manufacturing_director'
 'research_scientist' 'healthcare_representative' 'laboratory_technician'
 'sales_representative' 'human_resources']

Los valore únicos de la columna 'MaritalStatus' son: [nan 'married' 'divorced' 'single' 'marreid']



- En algunos valores de las columnas categóricas, por ejemplo, en la columna `MaritalStatus` en vez de "Married" en algunas filas aparece "Marreid". También aparecía 'Divorced' y 'divorced' como valores diferentes, pero con la función anterior ya se ha solucionado.

In [63]:
print(f"Los valores únicos de la columna 'MaritalStatus' son: {df_limpia['MaritalStatus'].unique()}")

Los valores únicos de la columna 'MaritalStatus' son: [nan 'married' 'divorced' 'single' 'marreid']


In [64]:
# Uso el método replace y compruebo que se ha cambiado
df_limpia['MaritalStatus'] = df_limpia['MaritalStatus'].replace('marreid', 'married')
print(f"Después de la limpieza, los valores únicos de la columna 'MaritalStatus' son: {df_limpia['MaritalStatus'].unique()}")

Después de la limpieza, los valores únicos de la columna 'MaritalStatus' son: [nan 'married' 'divorced' 'single']


## ÚLTIMO PASO

Guardamos el DataFrame con datos transformados en un nuevo archivo CSV.

In [66]:
# Guardamos el DataFrame limpio en un nuevo archivo CSV
output_file_path = 'hr_data_limpio.csv'
df_limpia.to_csv(output_file_path, index=False)
print(f"El DataFrame modificado ha sido guardado en '{output_file_path}'.")

El DataFrame modificado ha sido guardado en 'hr_data_limpio.csv'.
